# Settings

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import sys
import re
import pandas as pd
import requests

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

In [ ]:
# Selenium 설치
!pip install selenium
!pip install webdriver_manager


# Chrome WebDriver 설치
!apt-get update
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 341 kB in 4s (80.0 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

service = Service()

# Chrome WebDriver 실행
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 화면 출력 없이 실행하려면 이 옵션을 사용
options.add_argument('--no-sandbox')  # Colab 환경에서 실행할 때 필요한 옵션
options.add_argument('--disable-dev-shm-usage')  # Colab 환경에서 실행할 때 필요한 옵션
driver = webdriver.Chrome(service=service, options=options)  # 'chromedriver'는 PATH에 있는 경우 생략 가능

# 버전 확인
print("Chrome WebDriver Version:", driver.capabilities['chrome']['chromedriverVersion'])

# WebDriver 종료
driver.quit()


Chrome WebDriver Version: 116.0.5845.96 (1a391816688002153ef791ffe60d9e899a71a037-refs/branch-heads/5845@{#1382})


# Pre-Crawling

## 청약 전 날짜 리스트 + 공모주명 리스트

- 청약 전 날짜 데이터 및 공모주명 리스트는 38공모주(http://www.38.co.kr/html/fund/?o=k)를 참고하여 직접 datatime 및 문자열 리스트로 생성하였습니다.
- `start_list_n` / `ipo_list_n`에서 n은 38공모주의 공모주 청약일정 페이지 번호를 나타냅니다.
- 최신순으로 총 15페이지, 15*30=450개의 공모주 데이터입니다.

In [ ]:
# 청약 전 일 입력

start_list_1 = [datetime(2023, 9, 3), datetime(2023, 8, 28), datetime(2023, 8, 28), datetime(2023, 8, 22), datetime(2023, 8, 21), datetime(2023, 8, 20), datetime(2023, 8, 20), datetime(2023, 8,13),
              datetime(2023, 8, 9), datetime(2023, 8, 8), datetime(2023, 8, 6), datetime(2023, 7,31), datetime(2023, 7, 31), datetime(2023, 7, 31), datetime(2023, 7, 31), datetime(2023, 7, 30),
              datetime(2023, 7, 26), datetime(2023, 7, 25), datetime(2023, 7, 23), datetime(2023, 7, 16), datetime(2023, 7, 16), datetime(2023, 7, 16), datetime(2023, 7, 16), datetime(2023, 7, 12),
              datetime(2023, 7, 10), datetime(2023, 7, 9), datetime(2023, 7, 9), datetime(2023, 7, 4), datetime(2023, 7, 2), datetime(2023, 6, 26)] # 1페이지

start_list_2 = [datetime(2023, 6, 25), datetime(2023, 6, 20), datetime(2023, 6, 19), datetime(2023, 6, 19), datetime(2023, 6, 18), datetime(2023, 6, 12), datetime(2023, 6, 12), datetime(2023, 6,12),
              datetime(2023, 6, 6), datetime(2023, 6, 4), datetime(2023, 5, 24), datetime(2023, 5, 21), datetime(2023, 5, 21), datetime(2023, 5, 14), datetime(2023, 5, 9), datetime(2023, 5, 8),
              datetime(2023, 5, 7), datetime(2023, 5, 7), datetime(2023, 4, 23), datetime(2023, 4, 17), datetime(2023, 4, 16), datetime(2023, 3, 27), datetime(2023, 3, 26), datetime(2023, 3, 26),
              datetime(2023, 3, 26), datetime(2023, 3, 20), datetime(2023, 3, 19), datetime(2023, 3, 12), datetime(2023, 3, 6), datetime(2023, 3, 5)] # 2페이지

start_list_3 = [datetime(2023,3, 1), datetime(2023,2, 26), datetime(2023,2, 21), datetime(2023,2, 21), datetime(2023,2, 19), datetime(2023,2,19), datetime(2023,2, 19), datetime(2023,2, 15),
               datetime(2023,2, 13), datetime(2023,2, 8), datetime(2023,2, 6), datetime(2023,2, 5), datetime(2023,1, 30), datetime(2023,1, 25), datetime(2023,1, 24), datetime(2023,1, 15),
               datetime(2023,1, 15), datetime(2023,1, 9), datetime(2023,1,9), datetime(2022,12,14), datetime(2022,12,12), datetime(2022,12,11), datetime(2022,12,5), datetime(2022,12,5),
               datetime(2022,11,23), datetime(2022,11,23), datetime(2022,11,21), datetime(2022,11,20), datetime(2022,11,16), datetime(2022,11,14)] # 3페이지

start_list_4 = [datetime(2022, 11, 10), datetime(2022, 11, 8), datetime(2022, 11, 7), datetime(2022, 11, 7), datetime(2022, 11, 6), datetime(2022, 11, 1), datetime(2022, 11, 1), datetime(2022, 10, 25),
              datetime(2022, 10, 25), datetime(2022, 10, 23), datetime(2022, 10, 23), datetime(2022, 10, 23), datetime(2022, 10, 23), datetime(2022, 10, 18), datetime(2022, 10, 17), datetime(2022, 10, 17),
              datetime(2022, 10, 16), datetime(2022, 10, 11), datetime(2022, 10, 10), datetime(2022, 10, 10), datetime(2022, 10, 10), datetime(2022, 10, 10), datetime(2022, 10, 5), datetime(2022, 10, 4),
              datetime(2022, 10, 3), datetime(2022, 10, 3), datetime(2022, 9, 26), datetime(2022, 9, 25), datetime(2022, 9, 25), datetime(2022, 9, 23)] # 4페이지

start_list_5 = [datetime(2022, 9, 21), datetime(2022, 9, 19), datetime(2022, 9, 19), datetime(2022, 9, 19), datetime(2022, 9, 18), datetime(2022, 9, 14), datetime(2022, 9, 14), datetime(2022, 9, 12),
              datetime(2022, 8, 31), datetime(2022, 8, 28), datetime(2022, 8, 15), datetime(2022, 8, 15), datetime(2022, 8, 9), datetime(2022, 8, 8), datetime(2022, 7, 27), datetime(2022, 7, 25),
              datetime(2022, 7, 25), datetime(2022, 7, 19), datetime(2022, 7, 18), datetime(2022, 7, 18), datetime(2022, 7, 18), datetime(2022, 7, 17), datetime(2022, 7, 11), datetime(2022, 7, 11),
              datetime(2022, 7, 5), datetime(2022, 7, 4), datetime(2022, 7, 3), datetime(2022, 6, 26), datetime(2022, 6, 20), datetime(2022, 6, 20)] # 5페이지

start_list_6 = [datetime(2022, 6, 15), datetime(2022, 6, 13), datetime(2022, 6, 13), datetime(2022, 6, 13), datetime(2022, 6, 13), datetime(2022, 6, 13), datetime(2022, 6, 8), datetime(2022, 6, 7),
              datetime(2022, 5, 23), datetime(2022, 5, 16), datetime(2022, 5, 11), datetime(2022, 5, 10), datetime(2022, 5, 2), datetime(2022, 4, 18), datetime(2022, 4, 18), datetime(2022, 4, 17),
              datetime(2022, 4, 17), datetime(2022, 4, 10), datetime(2022, 3, 27), datetime(2022, 3, 21), datetime(2022, 3, 21), datetime(2022, 3, 14), datetime(2022, 3, 13), datetime(2022, 3, 6),
              datetime(2022, 3, 1), datetime(2022, 2, 24), datetime(2022, 2, 21), datetime(2022, 2, 20), datetime(2022, 2, 20), datetime(2022, 2, 16)] # 6페이지

start_list_7 = [datetime(2022, 2, 14), datetime(2022, 2, 14), datetime(2022, 2, 14), datetime(2022, 2, 13), datetime(2022, 2, 9), datetime(2022, 2, 8), datetime(2022, 2, 7), datetime(2022, 2, 6),
              datetime(2022, 1, 23), datetime(2022, 1, 23), datetime(2022, 1, 19), datetime(2022, 1, 19), datetime(2022, 1, 17), datetime(2022, 1, 12), datetime(2022, 1, 11), datetime(2022, 1, 10),
              datetime(2022, 1, 9), datetime(2021, 12, 19), datetime(2021, 12, 15), datetime(2021, 12, 15), datetime(2021, 12, 8), datetime(2021, 12, 8), datetime(2021, 12, 5), datetime(2021, 12, 1),
              datetime(2021, 11, 23), datetime(2021, 11, 21), datetime(2021, 11, 21), datetime(2021, 11, 10), datetime(2021, 11, 10), datetime(2021, 11, 8)] # 7페이지

start_list_8 = [datetime(2021,11, 7), datetime(2021,11, 2), datetime(2021,11, 1), datetime(2021,10, 31), datetime(2021,10, 31), datetime(2021,10, 31), datetime(2021,10, 26), datetime(2021,10, 24),
              datetime(2021,10, 24), datetime(2021,10, 20), datetime(2021,10, 19), datetime(2021,10, 18), datetime(2021,10, 17), datetime(2021,10, 11), datetime(2021,10, 11), datetime(2021,10, 4),
              datetime(2021,9, 29), datetime(2021,9, 27), datetime(2021,9,26), datetime(2021,9,26), datetime(2021,9,26), datetime(2021,9,22), datetime(2021,9,13), datetime(2021,9,13),
              datetime(2021,9,13), datetime(2021,9,12), datetime(2021,9,12), datetime(2021,9,6), datetime(2021,9,5), datetime(2021,9,1)] # 8페이지

start_list_9 = [datetime(2021,8, 30), datetime(2021,8, 29), datetime(2021,8, 29), datetime(2021,8, 23), datetime(2021,8, 23), datetime(2021,8, 11), datetime(2021,8, 8), datetime(2021,8, 8),
              datetime(2021,8, 8), datetime(2021,8, 4), datetime(2021,8, 4), datetime(2021,8, 4), datetime(2021,8, 3), datetime(2021,8, 3), datetime(2021,8, 1), datetime(2021,8, 1),
              datetime(2021,7, 28), datetime(2021,7, 25), datetime(2021,7, 25), datetime(2021,7, 18), datetime(2021,7, 15), datetime(2021,7, 12), datetime(2021,7, 7), datetime(2021,6, 30),
              datetime(2021,6, 14), datetime(2021,6, 8), datetime(2021,6, 7), datetime(2021,6, 6), datetime(2021,6, 1), datetime(2021,5, 24)] # 9페이지

start_list_10 = [datetime(2021,5, 12), datetime(2021,5, 12), datetime(2021,5, 10), datetime(2021,5, 10), datetime(2021,5, 10), datetime(2021,5, 9), datetime(2021,5, 5), datetime(2021,5, 2),
              datetime(2021,4, 27), datetime(2021,4, 27), datetime(2021,4, 18), datetime(2021,4, 11), datetime(2021,4, 11), datetime(2021,3, 29), datetime(2021,3, 24), datetime(2021,3, 21),
              datetime(2021,3, 14), datetime(2021,3, 14), datetime(2021,3, 10), datetime(2021,3, 8), datetime(2021,3, 7), datetime(2021,3, 4), datetime(2021,3, 3), datetime(2021,3, 1),
              datetime(2021,3, 1), datetime(2021,2, 24), datetime(2021,2, 22), datetime(2021,2, 17), datetime(2021,2, 15), datetime(2021,2, 14)] # 10페이지

start_list_11 = [datetime(2021, 2, 14), datetime(2021, 2, 14), datetime(2021, 2, 2),datetime(2021, 1, 28), datetime(2021, 1, 26),
              datetime(2021, 1, 24), datetime(2021, 1, 24), datetime(2021, 1, 24),datetime(2021, 1, 24), datetime(2021, 1, 20),
              datetime(2021, 1, 20), datetime(2021, 1, 18), datetime(2021, 1, 18),datetime(2021, 1, 17), datetime(2021, 1, 11),
              datetime(2021, 1, 11), datetime(2020, 12, 14), datetime(2020, 12, 13),datetime(2020, 12, 13), datetime(2020, 12, 13),
              datetime(2020, 12, 10), datetime(2020, 12, 9),datetime(2020, 12, 7), datetime(2020, 12, 6), datetime(2020, 12, 3),
              datetime(2020, 12, 2), datetime(2020, 12, 1),datetime(2020, 11, 30), datetime(2020, 11, 29), datetime(2020, 11, 29) ] # 11페이지

start_list_12 = [datetime(2020, 11, 26), datetime(2020, 11, 22), datetime(2020, 11, 22),datetime(2020, 11, 22), datetime(2020, 11, 22),
              datetime(2020, 11, 16), datetime(2020, 11, 12), datetime(2020, 11, 11), datetime(2020, 11, 9), datetime(2020, 11, 8),
              datetime(2020, 11, 8), datetime(2020, 11, 2), datetime(2020, 10, 28), datetime(2020, 10, 19), datetime(2020, 10, 19),
              datetime(2020, 10, 18), datetime(2020, 10, 12), datetime(2020, 10, 6), datetime(2020, 10, 4), datetime(2020, 9, 23),
              datetime(2020, 9, 14), datetime(2020, 9, 13), datetime(2020, 9, 13), datetime(2020, 9, 9), datetime(2020, 9, 8),
              datetime(2020, 9, 8), datetime(2020, 9, 6), datetime(2020, 9, 6), datetime(2022, 9, 2), datetime(2022, 8, 31) ] # 12페이지

start_list_13 = [datetime(2020, 8, 30), datetime(2020, 8, 17), datetime(2020, 8, 10),datetime(2020, 8, 9), datetime(2020, 8, 4),
              datetime(2020, 8, 4), datetime(2020, 8, 3), datetime(2020, 8, 2), datetime(2020, 7, 28), datetime(2020, 7, 26),
              datetime(2020, 7, 21), datetime(2020, 7, 20), datetime(2020, 7, 20), datetime(2020, 7, 19), datetime(2020, 7, 15),
              datetime(2020, 7, 14), datetime(2020, 7, 13), datetime(2020, 7, 13), datetime(2020, 7, 12), datetime(2020, 7, 12),
              datetime(2020, 7, 12), datetime(2020, 7, 7), datetime(2020, 7, 5), datetime(2022, 7, 5), datetime(2022, 6, 28),
              datetime(2020, 6, 24), datetime(2020, 6, 24), datetime(2020, 6, 22), datetime(2020, 6, 17), datetime(2020, 6, 15) ] # 13페이지

start_list_14 = [datetime(2020, 6, 14), datetime(2020, 6, 10), datetime(2020, 6, 10),datetime(2020, 6, 7), datetime(2020, 6, 7),
              datetime(2020, 5, 11), datetime(2020, 5, 10), datetime(2020, 3, 11), datetime(2020, 3, 2), datetime(2020, 3, 1),
              datetime(2020, 3, 1), datetime(2020, 2, 25), datetime(2020, 2, 23), datetime(2020, 2, 19), datetime(2020, 2, 18),
              datetime(2020, 2, 9), datetime(2020, 2, 2), datetime(2020, 1, 28), datetime(2020, 1, 15), datetime(2020, 1, 15),
              datetime(2019, 12, 16), datetime(2019, 12, 15), datetime(2019, 12, 15), datetime(2019, 12, 11), datetime(2019, 12, 11),
              datetime(2019, 12, 11),datetime(2019, 12, 10), datetime(2019, 12, 9), datetime(2019, 12, 8), datetime(2019, 12, 8) ]  # 14페이지

start_list_15 = [datetime(2019, 12, 8), datetime(2019, 12, 1), datetime(2019, 11, 27), datetime(2019, 11, 25), datetime(2019, 11, 24),
              datetime(2019, 11, 24),datetime(2019, 11, 20), datetime(2019, 11, 20), datetime(2019, 11, 17), datetime(2019, 11, 17),
              datetime(2019, 11, 13), datetime(2019, 11, 12), datetime(2019, 11, 10), datetime(2019, 11, 10), datetime(2019, 11, 10),
              datetime(2019, 11, 10),datetime(2019, 11, 6), datetime(2019, 11, 6), datetime(2019, 11, 5), datetime(2019, 11, 3),
              datetime(2019, 10, 30), datetime(2019, 10, 29), datetime(2019, 10, 27), datetime(2019, 10, 27), datetime(2019, 10, 22),
              datetime(2019, 10, 22),datetime(2019, 10, 20), datetime(2019, 10, 20), datetime(2019, 10, 20), datetime(2019, 10, 20) ] # 15페이지


In [ ]:
# 공모주 이름 리스트


ipo_list_1 = ['상상인스펙4호', '인스웨이브시스템즈', '한화플러스스팩4호', '대신밸런스스팩16호', '유안타스팩11호', '대신밸런스스팩15호', '한국스팩12호', '시큐레터', '스마트레이더시스템', '넥스틸', '빅토리콘텐츠', '큐리옥스바이오시스템즈', 'SK증권스팩10호',
             '코츠테크놀로지', '하나스팩28호', 'KB스팩26호', '파두', '엠아이큐브솔루션', '시지트로닉스', '파로스아이바이오', '버넥트', '에이엘티', '유안타스팩14호', '뷰티스킨', 'SK증권스팩9호', '센서뷰',
             '와이랩', '필에너지', '디비금융스팩11호', '이노시뮬레이션'] # 1페이지

ipo_list_2 = ['교보스팩14호', '오픈놀', '시큐센', '알멕', '하나스팩29호', '하이스팩8호', 'KB스팩25호', 'NH스팩29호', '프로테옴텍', '큐라티스', '마녀공장', '나라셀라',
            '진영(구.진영엘디엠)', '기가비스', '모니터랩', '씨유박스', '키움스팩8호', '트루엔', '에스바이오메딕스', '토마토시스템', '마이크로투나노', '미래에셋비전스팩3호', '삼성FN리츠(유가)', 'BKS스팩22호', '하나스팩27호',
            '지아이이노베이션', '엘비인베스트먼트', '한화위탁관리부동산투자회사', '유안타스팩12호', '미래에셋드림스팩1호'] # 2페이지

ipo_list_3 = ['금양그린파워', 'NH스팩28호', '자람테크놀로지', '하나스팩26호', '나노팀', '삼성스팩8호', '바이오인프라', '유안타스팩13호', '미래에셋비전스팩2호', '이노진', '제이오', '샌즈랩', '꿈비', '스튜디오미르',
             '삼기이브이', '오브젠', '미래반도체', '한주라이트메탈', '티이엠씨', '신영스팩9호', '바이오노트', '비엔케이스팩1호', 'IBKS스팩21호', 'NH스팩27호', '에스에이엠지엔터테인먼트', '대신밸런스스팩14호', '유진스팩9호', 'NH스팩26호',
              '대신밸런스스팩13호', '펨트론'] # 3페이지

ipo_list_4 = ['인벤티지랩', '엔젯', '티에프이', '유비온', '티쓰리엔터테인먼트', '윤성에프앤씨', '디티앤씨알오', '뉴로메카', '제이아이테크', 'NH스팩25호', '큐알티', '교보스팩13호', 'KB스팩23호', '저스템',
            '산돌', 'IBKS스팩20호', '삼성스팩7호', '플라즈맵', '한국스팩11호', '하나금융스팩25호', '핀텔', 'NH스팩24호', '샤페론', '에스비비테크', '오에스피', '탑머티리얼', '이노룰스',
            'SK증권스팩8호', '모델솔루션', '유안타스팩10호'] # 4페이지

ipo_list_5 = ['선바이오', '더블유씨피', '알피바이오', '한화플러스스팩3호', '하나금융스팩24호', '오픈엣지테크놀로지', 'KB스타리츠', 'KB스팩22호', '하나금융스팩23호', '키움스팩7호', 'IBKS스팩19호', '신영스팩8호',
            '쏘카', '대성하이텍', '에이치와이티씨', '새빗켐', '유안타스팩9호', '수산인더스트리', '에이프릴바이오', '신한스팩10호', '아이씨에이치', '성일하이텍', 'IBKS스팩18호', '루닛', '에이치피에스피',
            '대신밸런스12호스팩', '영창케미칼', '코난테크놀로지', '삼성스팩6호', '넥스트칩'] # 5페이지

ipo_list_6 = ['NH스팩23호', 'KB스팩21호', '교보스팩12호', '보로노이', '위니아에이드', '레이저쎌', '비플라이소프트', '범한퓨얼셀', '청담글로벌', '하나금융스팩22호', '마스턴프리미어리츠', '가온칩스',
            '대명에너지', '상상인스팩3호', '포바이포', '신영스팩7호', '신한스팩9호', '미래에셋비전스팩1호', '키움스팩6호', '지투파워', '유진스팩8호', '세아메카닉스', '공구우먼', '유일로보틱스', '코람코더원리츠',
            '모아데이타', 'SK증권스팩7호', '노을', '비씨엔씨', '풍원정밀'] # 6페이지

ipo_list_7 = ['브이씨', '스톤브릿지벤처스', '하나금융스팩21호', '퓨런티어', '한국스팩10호', '바이오에프디엔씨', 'IBKS스팩17호', '인카금융서비스', '나래나노텍', '아셈스', '스코넥', '이지트로닉스', 'LG에너지솔루션',
            '애드바이오텍', '케이옥션', '오토앤', '디비금융스팩10호', '래몽래인', '하나금융스팩20호', 'NH스팩22호', '하이스팩7호', '교보스팩11호', '케이티비네트워크', '툴젠', '신한서부티엔디리츠', '대신밸런스스팩11호',
            '미래에셋글로벌리츠', '알비더블유', '마인즈랩', '바이옵트로'] # 7페이지

ipo_list_8 = ['트윔', 'NH올원리츠', '지오엘리먼트', '비트나인', '디어유', '아이티아이즈', '지니너스', '카카오페이', '피코그램', '엔켐', '리파인', '지앤비에스엔지니어링', '아이패밀리에스씨', '차백신연구소',
            '지아이텍', 'NH스팩21호', '케이카', '씨유테크', '원준', '하나금융스팩19호', '아스플로', 'NH스팩20호', '에스앤디', '실리콘투', '프롬바이오', '신한스팩8호', '바이오플러스', '현대중공업',
             '대신밸런스스팩10호', '유진스팩7호'] # 8페이지

ipo_list_9 = ['와이엠텍', '에이비온', 'SK리츠', '일진하이솔루스', 'IBKS스팩16호', '바이젠셀', '브레인즈컴퍼니', '아주스틸', '롯데렌탈', '딥노이드', '한컴라이프케어', '디앤디플랫폼리츠', '플래티어', '엠로', '크래프톤',
            '원티드랩', '에이치케이이노엔', '카카오뱅크', '한화플러스스팩2호', '에브리봇', '맥스트', '큐라클', '에스디바이오센서', '오비고', '아모센스', '이노뎁', '삼성머스트스팩5호', '라온테크', '엘비루셈',
            '에이디엠코리아'] # 9페이지

ipo_list_10 = ['제주맥주', '진시스템', '삼영에스앤씨', '삼성스팩4호', 'NH스팩19호', '샘씨엔에스', '씨앤씨인터내셔널', '에이치피오', '하이스팩6호', 'SK아이이테크놀로지', '쿠콘', '해성티피씨', '이삭엔지니어링',
            '유진스팩6호', '유안타스팩8호', '엔시스', '자이언트스텝', '제노코', '라이프시맨틱스', 'SK바이오사이언스', '바이오다인', '디비금융스팩9호', '네오이뮨텍', '프레스티지바이오로직스', '싸이버원',
            '나노씨엠에스', '하나머스트스팩7호', 'IBKS스팩15호', '뷰노', '씨이랩'] # 10페이지

ipo_list_11 = ['유일에너테크', '오로스테크놀로지', '피엔에이치테크', '하나금융스팩17호', '아이퀘스트',
            '프레스티지바이오파마', '와이더플래닛', '레인보우로보틱스', '신한스팩7호', '솔루엠',
            '핑거', '씨앤투스성진', '모비릭스','선진뷰티사이언스', '엔비티',
            '한국스팩9호', '석경에이티', '지놈앤컴퍼니', '프리시젼바이오', '대신밸런스스팩9호',
            '엔에이치스팩18호', '알체라', '에프앤가이드', '에이치엠씨아이비스팩5호', '이에스알켄달스퀘어리츠',
            '티엘비', '인바이오', '엔젠바이오', '퀀타매트릭스', '디비금융스팩8호'] # 11페이지

ipo_list_12 = ['명신산업', '엔에프씨', '앱코', '포인트모바일', '클리노믹스',
            '제일전기공업', '하나기술', '티앤엘', '에이플러스에셋어드바이저', '네패스아크',
            '고바이오랩', '교촌에프앤비','소룩스', '위드텍', '센코',
            '바이브컴퍼니', '미코바이오메드', '피플바이오', '빅히트', '넥스틴',
            '원방테크', '엔에이치스팩17호', '비나텍', '박셀바이오', '비비씨',
            '핌스', '교보스팩10호', '압타머사이언스', '이오플로우', '카카오게임즈'] # 12페이지


ipo_list_13 = ['피엔케이피부임상연구센타', '미래에셋대우스팩5호', '아이디피', '셀레믹스',
            '미투젠', '코람코에너지플러스리츠', '브랜드엑스코퍼레이션', '영림원소프트랩', '한국파마',
            '이루다', '제이알글로벌리츠','이엔드디', '와이팜', '에이치엠씨아이비스팩4호',
            '엠투아이', '더네이쳐홀딩스', '하나금융스팩16호', '제놀루션', '솔트룩스',
            '티에스아이', '미래에셋맵스리츠1호', '에이프로', 'IBKS스팩13호', '이지스레지던스리츠',
            '소마젠', '위더스제약', '신도기연', 'SK바이오팜', '마크로밀엠브레인','이지스밸류플러스리츠'] # 13페이지


ipo_list_14 =  ['젠큐릭스', 'IBKS스팩14호', '엘이티', '에스씨엠생명과학', '엔에이치스팩16호',
            '드림씨아이에스', '이베스트스팩5호', '케이프이에스스팩4호', '엔피디', '플레이디',
            '에스케이스팩6호', '서울바이오시스','제이앤티씨', '켄코아에어로스페이스', '레몬',
            '서남', '신영스팩6호', '위세아이텍', '하나금융스팩15호', '케이비스팩20호',
            '천랩', '피피아이', '한화플러스스팩1호', '브릿지바이오테라퓨틱스', '엔에이치스팩15호',
            '메탈라이프', '하이스팩5호', '메드팩토', '신테카바이오', '대신밸런스스팩8호'] # 14페이지


ipo_list_15 = ['유안타스팩6호', '제이엘케이인스펙션', 'IBKS스팩12호', '태웅로직스', '에스케이스팩5호',
            '리메드', '코리아센터', 'IBKS스팩11호', '유안타스팩5호', '엔에이치프라임리츠',
            '노터스', '센트랄모텍', '티움바이오', '코리아에셋투자증권', '씨에스베어링',
            '우양', '에스제이그룹', '현대에너지솔루션', '제테마', '한화시스템',
            '라파스', '아이티엠반도체', '자이에스앤디', '미래에셋대우스팩4호', '하나금융스팩14호',
            '티라유텍', '미디어젠', '신한스팩6호', '캐리소프트','지누스'] # 15페이지

In [ ]:
'''
청약 D-1로부터 D-30 날짜 계산하기
      - result_list : 각 공모주 별 (청약 D-1 날짜, 청약 D-30 날짜)
      - start_date : 청약 D-1
      - end_date   : 청약 D-30
'''

# 결과를 저장할 리스트 초기화
for a in range(15):
    globals()[f"result_list_{a+1}"] = []
    print(f"변수 초기화 완료:{a+1}")

for b in range(15):
  # 각각의 시작 날짜에 대해 반복문 실행
  for start_date in globals()[f"start_list_{b+1}"]:
    # 시작 날짜로부터 한달 전 날짜 계산
    end_date = start_date - timedelta(days=30)

    # 결과를 리스트에 추가 (데이터프레임에 추가하지 않고 튜플로 저장)
    globals()[f"result_list_{b+1}"].append((start_date.strftime('%Y. %m. %d.'), end_date.strftime('%Y. %m. %d.')))

for j in range(15):
  print(f'result_list_{j+1} 시작 !@!')
  # 결과 출력
  for start_date, end_date in globals()[f"result_list_{j+1}"]:
      print(f"Start Date: {start_date}, End Date: {end_date}")

변수 초기화 완료:1
변수 초기화 완료:2
변수 초기화 완료:3
변수 초기화 완료:4
변수 초기화 완료:5
변수 초기화 완료:6
변수 초기화 완료:7
변수 초기화 완료:8
변수 초기화 완료:9
변수 초기화 완료:10
변수 초기화 완료:11
변수 초기화 완료:12
변수 초기화 완료:13
변수 초기화 완료:14
변수 초기화 완료:15
result_list_1 시작 !@!
Start Date: 2023. 09. 03., End Date: 2023. 08. 04.
Start Date: 2023. 08. 28., End Date: 2023. 07. 29.
Start Date: 2023. 08. 28., End Date: 2023. 07. 29.
Start Date: 2023. 08. 22., End Date: 2023. 07. 23.
Start Date: 2023. 08. 21., End Date: 2023. 07. 22.
Start Date: 2023. 08. 20., End Date: 2023. 07. 21.
Start Date: 2023. 08. 20., End Date: 2023. 07. 21.
Start Date: 2023. 08. 13., End Date: 2023. 07. 14.
Start Date: 2023. 08. 09., End Date: 2023. 07. 10.
Start Date: 2023. 08. 08., End Date: 2023. 07. 09.
Start Date: 2023. 08. 06., End Date: 2023. 07. 07.
Start Date: 2023. 07. 31., End Date: 2023. 07. 01.
Start Date: 2023. 07. 31., End Date: 2023. 07. 01.
Start Date: 2023. 07. 31., End Date: 2023. 07. 01.
Start Date: 2023. 07. 31., End Date: 2023. 07. 01.
Start Date: 2023. 07. 30., E

## 날짜 필터링 함수

In [ ]:
def convert_relative_date(relative_date):
  """들어온 날짜 데이터에 문자열이 포함되어 있다면 날짜 형식으로 변환해준다.

    Args:
        relative_date: 기사 작성일.

    Returns:
        통일된 날짜 형식으로 반환.
    """
  if '어제' in relative_date:
    current_date = datetime.today()
    modified_date = current_date - timedelta(days=1)
    return modified_date.strftime('%Y. %m. %d.')

  elif '일 전' in relative_date:
    days_ago = int(relative_date.split('일 전')[0])
    current_date = datetime.today()
    modified_date = current_date - timedelta(days=days_ago)
    return modified_date.strftime('%Y. %m. %d.')

  elif '시간 전' in relative_date:
    hours_ago = int(relative_date.split('시간 전')[0])
    current_date = datetime.today()
    modified_date = current_date - timedelta(hours=hours_ago)
    return modified_date.strftime('%Y. %m. %d.')

  else:
    return relative_date


## 언론사별

###1) 머니투데이

In [ ]:
def mt(link):
  response = requests.get(link)
  soup=BeautifulSoup(response.text,'html.parser')
  title=soup.find('h1',{'class':'subject'})
  if title is None:
    title=''
  else:
    title=title.text

  text=soup.find('div',{'id':'textBody'})
  if text is None:
    text=''
  else:
    text=text.text
  return title, text


###2)매일경제

In [ ]:
def mk(link):
  response = requests.get(link)
  response.encoding = 'utf-8'  # 지정된 인코딩으로 설정
  soup = BeautifulSoup(response.text,'html.parser')
  title=soup.find('h2',{'class':'news_ttl'})
  if title is None:
    title = soup.find('div',{'class':'news_title_text'})
    if title is None:
      title = ''
    else:
      title = title.find('h1', {'class':'top_title'}).text
  else:
    title=title.text

  content = soup.find('div',{'class':'news_cnt_detail_wrap'})
  if content is None:
    # 예외처리 시작
    content = soup.find('div',{'itemprop':'articleBody'})
    if content is None:
      content = ''
    else:
      content = content.text
    # 예외처리 끝
  else:
    content = content.text
  return title, content

###3)서울경제

In [ ]:
def sedaily(link):
  response = requests.get(link)
  soup=BeautifulSoup(response.text,'html.parser')
  title=soup.find('h1',{'class':'art_tit'})
  if title is None:
    title=soup.find('div',{'class':'headline'}).find('h1')
    if title is None:
      title = ''
    else:
      title = title.text
  else:
    title=title.text

  content=soup.find('div',{'class':'article_view'})
  if content is None:
    content=soup.find('div',{'itemprop':'articleBody'})
    if content is None:
      content = ''
    else:
      content = content.text
  else:
    content=content.text
  return title, content

###4)더벨

In [ ]:
def thebell(link):
  response = requests.get(link)
  soup=BeautifulSoup(response.text,'html.parser')
  title=soup.find('p',{'class':'tit'})
  if title is None:
    title=''
  else:
    title=title.text

  content=soup.find('div',{'class':'viewSection'})
  if content is None:
    content=''
  else:
    content=content.text[60:]
  return title, content

###5)한국경제

In [ ]:
def hankyung(link):
  driver = webdriver.Chrome(options=options)
  driver.get(link)
  driver.implicitly_wait(3)
  try:
    title = driver.find_element('class name', 'article-contents')
    if title is None:
      title=''
    else:
      title=title.find_element('class name', 'headline').text
  except:
    title = driver.find_element('class name', 'article-tit').text

  text = driver.find_element('class name', 'article-body').text
  return title, text

###6)이투데이

In [ ]:
def etoday(link):
  response = requests.get(link)
  soup=BeautifulSoup(response.text,'html.parser')
  title=soup.find('h1',{'class':'main_title'})
  if title is None:
    title=''
  else:
    title=title.text

  text=soup.find('div',{'class':'articleView'})
  content_list=[]
  if text is None:
    text=''
  else:
    text=text.find_all('p')
    for i in range(len(text)):
      content_list.append(text[i].text)
    text=" ".join(content_list) #본문
  return title, text


###7)머니s

In [ ]:
def moneys(link):
  response = requests.get(link)
  soup=BeautifulSoup(response.text,'html.parser')
  title=soup.find('div',{'id':'article'}).find('h1')
  if title is None:
    title=soup.find('h1',{'class':'mgt37'})
    if title is None:
      title=soup.find('h2',{'class':'ellipsis title'})
      if title is None:
        title = ''
      else:
        title = title.text
    else:
      title = title.text
  else:
    title=title.text

  content=soup.find('div',{'itemprop':'articleBody'})
  if content is None:
    content=soup.find('article',{'class':'article_body'})
    if content is None:
      content = ''
    else:
      content = content.text
  else:
    content=content.text
  return title, content


###8)아시아 경제

In [ ]:
def asiae(link):
  response = requests.get(link)
  soup=BeautifulSoup(response.text,'html.parser')
  title=soup.find('h1',{'class':'mainTitle'})
  if title is None:
    # 예외
    title=soup.find('div', {'class':'area_title'})
    if title is None:
      title = ''
    else:
      title = title.find('h1').text
    # 예외 끝
  else:
    title=title.text

  text=soup.find('div',{'class':'va_cont'})
  content_list=[]
  if text is None:
    # 예외
    text=soup.find('div',{'itemprop':'articleBody'})
    if text is None:
      text=''
    else:
      text=text.find_all('p')
      content_list=[]
      for i in range(len(text)):
        content_list.append(text[i].text)
      text=" ".join(content_list) #본문
    # 예외 끝
  else:
    text=text.find_all('p')
    content_list=[]

    for i in range(len(text)):
      content_list.append(text[i].text)
    text=" ".join(content_list) #본문
  return title, text


###9)서울파이낸스

In [ ]:
def seoulfn(link):
  response = requests.get(link)
  soup=BeautifulSoup(response.text,'html.parser')
  title=soup.find('div',{'class':'article-head-title'})
  if title is None:
    title=''
  else:
    title=title.text

  text=soup.find('div',{'id':'article-view-content-div'})
  content_list=[]
  if text is None:
    text=''
  else:
    text=text.find_all('p')

    for i in range(len(text)):
      content_list.append(text[i].text)
    text=" ".join(content_list) #본문
  return title,  text


###10)딜사이트

In [ ]:
def dealsite(link):
  response = requests.get(link)
  soup=BeautifulSoup(response.text,'html.parser')
  title=soup.find('div',{'class':'read-news-title'})
  if title is None:
    title=''
  else:
    title=title.text

  text=soup.find('div',{'class':'rnmc-right rnmc-right1 content-area'})
  content_list=[]
  if text is None:
    text=''
  else:
    text=text.find_all('p')

    for i in range(len(text)):
      content_list.append(text[i].text)
    text=" ".join(content_list) #본문
  return title, text


###11)비즈워치

In [ ]:
def bizwatch(link):
  response = requests.get(link)
  soup=BeautifulSoup(response.text,'html.parser')
  title=soup.find('div',{'class':'top_content'}).find('h1')
  if title is None:
    title=''
  else:
    title=title.text

  text=soup.find('div',{'itemprop':'articleBody'})
  content_list=[]
  if text is None:
    text=''
  else:
    text=text.find_all('p')

    for i in range(len(text)):
      content_list.append(text[i].text)
    text=" ".join(content_list) #본문
  return title, text


###12)이코노미스트

In [ ]:
def economist(link):
  driver = webdriver.Chrome(options=options)
  driver.get(base_url)
  driver.get(link)
  driver.implicitly_wait(10)
  title = driver.find_element('id', 'article_body')
  if title is None:
    title=''
  else:
    title=title.find_element('class name', 'view-article-title').text

  text = driver.find_element('class name', 'content').text
  return title, text


###13)라이센스뉴스

In [ ]:
def license_news(link):
  driver = webdriver.Chrome(options=options)
  driver.get(base_url)
  driver.get(link)
  driver.implicitly_wait(10)
  title = driver.find_element('class name', 'article-view-header')
  if title is None:
    title=''
  else:
    title=title.find_element('class name', 'heading').text

  text = driver.find_element('id', 'article-view-content-div').text
  return title, text


###14)시사저널e

In [ ]:
def sisa(link):
  driver = webdriver.Chrome(options=options)
  driver.get(base_url)
  driver.get(link)
  driver.implicitly_wait(10)
  title = driver.find_element('class name', 'article-header-wrap')
  if title is None:
    title=''
  else:
    title=title.find_element('class name', 'article-head-title').text

  text = driver.find_element('id', 'article-view-content-div').text
  return title, text


###15)충청신문

In [ ]:
def chungcheong(link):
  driver = webdriver.Chrome(options=options)
  driver.get(base_url)
  driver.get(link)
  driver.implicitly_wait(10)
  title = driver.find_element('class name', 'article-view-header')
  if title is None:
    title=''
  else:
    title=title.find_element('class name', 'heading').text

  text = driver.find_element('id', 'article-view-content-div').text
  return title, text


###16)조선비즈

In [ ]:
def chosun(link):
  driver = webdriver.Chrome(options=options)
  driver.get(base_url)
  driver.get(link)
  driver.implicitly_wait(10)
  title = driver.find_element('class name', 'article-header__headline')
  if title is None:
    title=''
  else:
    title=title.text

  text = driver.find_element('class name', 'article-body').text
  return title, text


###17)전국매일신문

In [ ]:
def jkmail(link):
  response = requests.get(link)
  response.encoding = 'utf-8'  # 지정된 인코딩으로 설정
  soup=BeautifulSoup(response.text,'html.parser')

  title=soup.find('div',{'class':'article-header-wrap'})
  if title is None:
    title=''
  else:
    title=title.find('div', {'class':'article-head-title'}).text

  content=soup.find('div',{'itemprop':'articleBody'})
  if content is None:
    content=''
  else:
    content=content.text
  return title, content # title, content 완료

###18) Investing.com

In [ ]:
def inv(link):
  response = requests.get(link)
  response.encoding = 'utf-8'  # 지정된 인코딩으로 설정
  soup=BeautifulSoup(response.text,'html.parser')

  title=soup.find('h1',{'class':'articleHeader'})
  if title is None:
    title=''
  else:
    title=title.text

  content=soup.find('div',{'class':'WYSIWYG articlePage'})
  if content is None:
    content=''
  else:
    content=content.find_all('p')
    content_list=[]
    for i in range(len(content)):
      content_list.append(content[i].text)
    content=" ".join(content_list) #본문
  return title, content # title, content 완료

###19) 영남일보

In [ ]:
def yn(link):
  response = requests.get(link)
  response.encoding = 'utf-8'  # 지정된 인코딩으로 설정
  soup=BeautifulSoup(response.text,'html.parser')

  title=soup.find('p', {'class':'article-top-title'})
  if title is None:
    # 예외
    title=soup.find('h1', {'class':'article-top-title'})
    if title is None:
      title = ''
    else:
      title = title.text

  content=soup.find('div',{'itemprop':'articleBody'})
  if content is None:
    content=''
  else:
    content=content.text
  return title, content # title, content 완료

###20) CBC뉴스

In [ ]:
def cbc(link):
  response = requests.get(link)
  response.encoding = 'utf-8'  # 지정된 인코딩으로 설정
  soup=BeautifulSoup(response.text,'html.parser')

  title=soup.find('div',{'class':'article-header-wrap'})
  if title is None:
    title=''
  else:
    title=title.find('div', {'class':'article-head-title'}).text

  content=soup.find('div',{'itemprop':'articleBody'})
  if content is None:
    content=''
  else:
    content=content.find_all('p')
    content_list=[]
    for i in range(len(content)):
      content_list.append(content[i].text)
    content=" ".join(content_list) #본문
  return title, content # title, content 완료

###21) 블로터

In [ ]:
def blt(link):
  response = requests.get(link)
  response.encoding = 'utf-8'  # 지정된 인코딩으로 설정
  soup=BeautifulSoup(response.text,'html.parser')

  title=soup.find('header',{'class':'article-view-header'})
  if title is None:
    title=''
  else:
    title=title.find('h1', {'class':'heading'}).text

  content = soup.find('article',{'itemprop' : 'articleBody'})
  if content is None:
    content = ''
  else:
    content=content.text
  return title, content # title, content 완료

###22) 더팩트

In [ ]:
def thefact(link):
  response = requests.get(link)
  soup=BeautifulSoup(response.text,'html.parser')

  title=soup.find('h2',{'class':'articleTitle'})
  if title is None:
    title=''
  else:
    title=title.text

  content = soup.find('div',{'class' : 'mArticle'})
  if content is None:
    content=''
  else:
    content=content.find_all('p')
    content_list=[]
    for i in range(len(content)):
      content_list.append(content[i].text)
    content=" ".join(content_list) #본문
  return title, content # title, content 완료

###23) 아시아 에이

In [ ]:
def asiaa(link):
  response = requests.get(link)  # 지정된 인코딩으로 설정
  soup=BeautifulSoup(response.text,'html.parser')

  title=soup.find('h3',{'class':'heading'})
  if title is None:
    title=''
  else:
    title=title.text

  content = soup.find('article',{'id' : 'article-view-content-div'})
  if content is None:
    content=''
  else:
    content=content.find_all('p')
    content_list=[]
    for i in range(len(content)):
      content_list.append(content[i].text)
    content=" ".join(content_list) #본문
  return title, content # title, content 완료

###24) 문화뉴스

In [ ]:
def culture(link):
  response = requests.get(link)  # 지정된 인코딩으로 설정
  soup=BeautifulSoup(response.text,'html.parser')

  title=soup.find('h3',{'class':'heading'})
  if title is None:
    title=''
  else:
    title=title.text

  content = soup.find('div',{'style' : 'text-align:center'})
  if content is None:
    content=''
  else:
    content=content.find_all('p')
    content_list=[]
    for i in range(len(content)):
      content_list.append(content[i].text)
    content=" ".join(content_list) #본문
  return title, content # title, content 완료

###25) 한겨레

In [ ]:
def hangyere(link):
  response = requests.get(link)
  response.encoding = 'utf-8'  # 지정된 인코딩으로 설정
  soup=BeautifulSoup(response.text,'html.parser')

  title=soup.find('span',{'class':'title'})
  if title is None:
    title=''
  else:
    title=title.text

  content=soup.find('div',{'itemprop':'articleBody'})
  if content is None:
    content=''
  else:
    content=content.text
  return title, content


###26) 뉴스핌

In [ ]:
def pim(link):
  response = requests.get(link)  # 지정된 인코딩으로 설정
  soup=BeautifulSoup(response.text,'html.parser')

  title=soup.find('h2',{'id':'main-title'})
  if title is None:
    title=''
  else:
    title=title.text

  content = soup.find('div',{'id':'news-contents'})
  if content is None:
    content=''
  else:
    content=content.find_all('p')
    content_list=[]
    for i in range(len(content)):
      content_list.append(content[i].text)
    content=" ".join(content_list) #본문
  return title, content # title, content 완료

# Crawling

## 1. 기사 링크 리스트 크롤링

- 구글 뉴스에서 `'"각 공모주명"+"공모주"'`를 키워드로 설정하여 검색한 결과 속 각 신문사로 이어지는 뉴스 링크 리스트를 크롤링 하였습니다.
- `real_result` : 기사별 링크(link), 신문사(article), 관련 공모주(name), 날짜(date) 데이터 **프레임**

In [ ]:
real_result = pd.DataFrame(columns=['link','article','name','date'])

# 총 15페이지, 한 페이지당 공모주 30개씩 링크 리스트 가져오기
for j in range(15):
  # 한 페이지 속 모든 공모주 하나씩 꺼내기.
  for k in range(len(globals()[f"ipo_list_{j+1}"])):
    zzin_result=pd.DataFrame(columns=['index','link','article','name', 'date']) #전체 공모주 모아놓을 데이터프레임.
    stock = globals()[f"ipo_list_{j+1}"][k] # 공모주 이름
    keyword =f'"{stock}"%2B"공모주"' # 따옴표 사이에 공모주이름 껴놓기
    url = "https://news.google.com/search?q="+keyword+"&hl=ko&gl=KR&ceid=KR%3Ako"
    resp = requests.get(url)
    html_src = resp.text
    soup = BeautifulSoup(html_src, 'html.parser')
    news_items = soup.select('div.xrnccd')

    base_url = "https://news.google.com"
    links=[]; titles=[]; agencies=[]; reporting_dates=[]; reporting_times=[]; date=[];
    for item in news_items:
      # link
      link = item.find('a', attrs={'class':'VDXfz'}).get('href')
      news_link = base_url + link[1:]
      links.append(news_link)
      # article
      news_agency = item.find('div',attrs={'class':'N0NI1d AVN2gc WfKKme'}).text
      agencies.append(news_agency)
      # date
      date1 = item.find('div', attrs={'class':'QmrVtf RD0gLb kybdz'})
      date2 = date1.find('div', attrs={'class':'SVJrMe'})
      date3 = date2.find('time', attrs={'class':'WW6dff uQIVzc Sksgp slhocf'}).text
      date.append(date3)

    google_df=pd.DataFrame(columns=['link','article','name', 'date']) #공모주별로 일시적으로 담아줄 데이터프레임.
    for i in range(len(links)):
      google_df=google_df.append({'link':links[i],'article':agencies[i],'name':f'{stock}', 'date':date[i]},ignore_index=True)
    google_df=google_df.reset_index()
    zzin_result=pd.concat([zzin_result,google_df]) #전체 데이터프레임에 누적시키기

    # 날짜 형식 통일하기
    for idx, row in zzin_result.iterrows():
      row['date'] = convert_relative_date(row['date'])

    current_year = datetime.today().year
    for idx, row in zzin_result.iterrows():
      date_str = row['date']
      if '월' in date_str and '일' in date_str:
        month, day = date_str.split()[0].split('월')[0], date_str.split()[1].split('일')[0]
        modified_date = datetime(current_year, int(month), int(day))
        row['date'] = modified_date.strftime('%Y. %m. %d.')

    # 날짜 필터링
    start_date = globals()[f"result_list_{j+1}"][k][0] # 시작 날짜
    end_date = globals()[f"result_list_{j+1}"][k][1] # 종료 날짜
    # datetime 형식으로 변환 후 filtering
    zzin_result['date'] = pd.to_datetime(zzin_result['date'], format='%Y. %m. %d.')
    print("before", zzin_result['date'])
    # 청약 D-30 ~ D-1 안에 드는 데이터만 남기기
    zzin_result = zzin_result[(zzin_result['date'] >= end_date) & (zzin_result['date'] <= start_date)]
    print("after", zzin_result['date'])

    zzin_result.drop(['index'],axis=1, inplace=True) # index drop

    real_result=pd.concat([real_result,zzin_result],ignore_index=True) # real_result에 zzin_result 넣기

real_result.to_csv("[구글]1-15페이지.csv")

before Series([], Name: date, dtype: datetime64[ns])
after Series([], Name: date, dtype: datetime64[ns])
before 0    2023-08-13
1    2023-08-19
2    2023-08-04
3    2023-07-28
4    2023-08-01
5    2023-08-17
6    2023-08-01
7    2023-08-05
8    2023-06-01
9    2023-07-14
10   2023-07-21
11   2023-05-26
12   2023-06-09
13   2023-06-03
14   2023-06-30
15   2023-07-12
16   2023-07-19
17   2022-10-17
18   2022-11-04
19   2014-10-17
Name: date, dtype: datetime64[ns]
after 0   2023-08-13
1   2023-08-19
2   2023-08-04
4   2023-08-01
5   2023-08-17
6   2023-08-01
7   2023-08-05
Name: date, dtype: datetime64[ns]
before 0   2023-08-19
1   2023-08-21
2   2023-08-17
Name: date, dtype: datetime64[ns]
after 0   2023-08-19
1   2023-08-21
2   2023-08-17
Name: date, dtype: datetime64[ns]
before 0   2023-08-17
1   2023-08-19
2   2023-08-21
3   2023-08-13
Name: date, dtype: datetime64[ns]
after 0   2023-08-17
1   2023-08-19
2   2023-08-21
3   2023-08-13
Name: date, dtype: datetime64[ns]
before 0   2023-0

<ipython-input-13-17bba36324e8>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zzin_result.drop(['index'],axis=1, inplace=True)


before 0   2023-08-21
1   2023-08-19
2   2023-08-21
3   2023-08-13
Name: date, dtype: datetime64[ns]
after 1   2023-08-19
3   2023-08-13
Name: date, dtype: datetime64[ns]


<ipython-input-13-17bba36324e8>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zzin_result.drop(['index'],axis=1, inplace=True)


before 0    2023-08-12
1    2023-08-15
2    2023-08-16
3    2023-08-19
4    2023-08-20
        ...    
95   2022-07-03
96   2021-11-21
97   2023-07-06
98   2021-10-03
99   2023-03-14
Name: date, Length: 100, dtype: datetime64[ns]
after 0    2023-08-12
5    2023-08-11
7    2023-08-06
8    2023-07-31
9    2023-08-11
10   2023-08-09
12   2023-07-24
13   2023-07-24
14   2023-08-13
15   2023-08-03
16   2023-07-15
23   2023-07-16
24   2023-07-18
29   2023-07-15
34   2023-07-20
37   2023-07-21
56   2023-07-17
70   2023-07-19
76   2023-07-18
79   2023-07-19
80   2023-07-20
Name: date, dtype: datetime64[ns]
before 0    2023-08-20
1    2023-08-22
2    2023-08-21
3    2023-08-21
4    2023-08-09
        ...    
95   2023-01-06
96   2023-01-24
97   2023-02-16
98   2023-03-28
99   2023-01-12
Name: date, Length: 100, dtype: datetime64[ns]
after 4    2023-08-09
5    2023-08-09
9    2023-08-04
11   2023-08-09
15   2023-08-09
16   2023-07-24
18   2023-07-15
21   2023-07-16
23   2023-07-10
24   2023-07-1

...

before Series([], Name: date, dtype: datetime64[ns])
after Series([], Name: date, dtype: datetime64[ns])
before 0    2023-07-17
1    2022-10-05
2    2022-11-29
3    2020-10-23
4    2020-09-21
5    2020-12-22
6    2020-09-22
7    2021-05-31
8    2020-09-23
9    2022-03-28
10   2018-01-08
11   2020-09-22
12   2020-07-28
13   2020-04-09
14   2020-09-19
15   2020-04-07
16   2020-09-18
17   2020-11-25
18   2020-09-04
Name: date, dtype: datetime64[ns]
after 18   2020-09-04
Name: date, dtype: datetime64[ns]
before 0    2023-07-09
1    2020-07-21
2    2020-10-23
3    2020-09-01
4    2020-09-19
5    2023-05-22
6    2022-11-15
7    2020-09-22
8    2020-09-14
9    2020-09-21
10   2020-09-23
11   2020-07-29
12   2021-03-03
13   2021-10-29
14   2020-09-22
15   2021-12-20
16   2020-12-10
17   2021-01-04
18   2020-09-24
19   2020-09-18
20   2020-12-29
21   2020-07-22
Name: date, dtype: datetime64[ns]
after Series([], Name: date, dtype: datetime64[ns])
before 0    2023-07-24
1    2023-07-26
2    2020-

<ipython-input-13-17bba36324e8>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zzin_result.drop(['index'],axis=1, inplace=True)


before 0   2020-07-01
1   2020-06-04
2   2020-07-20
3   2020-06-25
4   2020-06-10
Name: date, dtype: datetime64[ns]
after 1   2020-06-04
4   2020-06-10
Name: date, dtype: datetime64[ns]
before 0   2020-06-17
1   2020-07-16
2   2022-05-05
3   2022-08-08
4   2022-01-03
5   2020-06-26
6   2020-09-01
7   2020-06-24
8   2020-07-13
9   2020-12-23
Name: date, dtype: datetime64[ns]
after Series([], Name: date, dtype: datetime64[ns])
before 0    2023-07-09
1    2023-06-21
2    2023-06-01
3    2023-05-26
4    2023-05-31
        ...    
95   2023-04-06
96   2023-04-02
97   2023-05-16
98   2023-05-18
99   2023-05-31
Name: date, Length: 100, dtype: datetime64[ns]
after 69   2020-06-09
Name: date, dtype: datetime64[ns]
before Series([], Name: date, dtype: datetime64[ns])
after Series([], Name: date, dtype: datetime64[ns])
before 0    2020-06-10
1    2020-06-05
2    2020-06-24
3    2020-07-20
4    2020-05-26
5    2020-12-19
6    2020-09-30
7    2020-06-16
8    2020-07-02
9    2020-07-05
10   2020-01-

<ipython-input-13-17bba36324e8>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zzin_result.drop(['index'],axis=1, inplace=True)


before 0    2023-07-07
1    2020-02-16
2    2023-06-28
3    2023-05-19
4    2020-08-15
        ...    
75   2019-01-21
76   2018-07-06
77   2019-06-03
78   2019-11-15
79   2022-08-27
Name: date, Length: 80, dtype: datetime64[ns]
after 1   2020-02-16
8   2020-02-14
Name: date, dtype: datetime64[ns]
before 0    2023-08-09
1    2023-07-08
2    2023-07-06
3    2023-07-07
4    2023-06-30
        ...    
95   2023-03-16
96   2023-03-22
97   2023-04-26
98   2023-03-02
99   2023-06-21
Name: date, Length: 100, dtype: datetime64[ns]
after 14   2020-02-07
17   2020-02-03
92   2020-01-28
Name: date, dtype: datetime64[ns]
before 0   2021-06-02
1   2021-05-30
Name: date, dtype: datetime64[ns]
after Series([], Name: date, dtype: datetime64[ns])
before 0    2023-08-12
1    2023-06-10
2    2023-04-20
3    2023-02-25
4    2023-05-02
5    2023-06-18
6    2023-06-07
7    2023-04-02
8    2023-01-13
9    2023-05-19
10   2023-02-03
11   2023-03-16
12   2023-02-08
13   2021-11-09
14   2020-06-24
15   2023-06-

In [ ]:
# 기사별 링크 크롤링 결과
real_result

,link,article,name,date
0,https://news.google.com/articles/CBMiOWh0dHBzO...,블로터,인스웨이브시스템즈,2023-08-13
1,https://news.google.com/articles/CBMiOWh0dHA6L...,비즈워치,인스웨이브시스템즈,2023-08-19
2,https://news.google.com/articles/CBMiJmh0dHBzO...,딜사이트,인스웨이브시스템즈,2023-08-04
3,https://news.google.com/articles/CBMiXWh0dHA6L...,더벨(thebell),인스웨이브시스템즈,2023-08-01
4,https://news.google.com/articles/CBMiOmh0dHBzO...,충청신문,인스웨이브시스템즈,2023-08-17
...,...,...,...,...
2338,https://news.google.com/articles/CBMiXGh0dHBzO...,더벨(thebell),티라유텍,2019-10-22
2339,https://news.google.com/articles/CBMiKmh0dHBzO...,이투데이,티라유텍,2019-10-16
2340,https://news.google.com/articles/CBMiKmh0dHBzO...,이투데이,미디어젠,2019-10-16
2341,https://news.google.com/articles/CBMiOmh0dHBzO...,서울파이낸스,미디어젠,2019-10-11


## 뉴스 필터링

In [ ]:
csv_file_path = f'/content/drive/MyDrive/[구글]1-15페이지.csv'
df = pd.read_csv(csv_file_path)
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df

,link,article,name,date
0,https://news.google.com/articles/CBMiOWh0dHBzO...,블로터,인스웨이브시스템즈,2023-08-13
1,https://news.google.com/articles/CBMiOWh0dHA6L...,비즈워치,인스웨이브시스템즈,2023-08-19
2,https://news.google.com/articles/CBMiJmh0dHBzO...,딜사이트,인스웨이브시스템즈,2023-08-04
3,https://news.google.com/articles/CBMiXWh0dHA6L...,더벨(thebell),인스웨이브시스템즈,2023-08-01
4,https://news.google.com/articles/CBMiOmh0dHBzO...,충청신문,인스웨이브시스템즈,2023-08-17
...,...,...,...,...
2338,https://news.google.com/articles/CBMiXGh0dHBzO...,더벨(thebell),티라유텍,2019-10-22
2339,https://news.google.com/articles/CBMiKmh0dHBzO...,이투데이,티라유텍,2019-10-16
2340,https://news.google.com/articles/CBMiKmh0dHBzO...,이투데이,미디어젠,2019-10-16
2341,https://news.google.com/articles/CBMiOmh0dHBzO...,서울파이낸스,미디어젠,2019-10-11


- values_to_keep : 크롤링 대상 언론사

In [ ]:
values_to_keep = ['머니투데이','매일경제','서울경제신문', '서울경제','더벨(thebell)', '더벨','한국경제',
                  '이투데이','머니s', '머니S','아시아경제','서울파이낸스','딜사이트',
                  '비즈워치', '이코노미스트','라이센스뉴스','시사저널e','충청신문',
                  '조선비즈','전국매일신문','Investing.com','영남일보','CBC뉴스',
                  '블로터','더팩트','아시아에이','문화뉴스','한겨레','뉴스핌',
                  '이데일리', '비즈니스포스트', '시사저널이코노미', '파이낸셜뉴스', '국제뉴스',
                  '데일리안', 'MTN', '아주경제', '뉴스토마토', '뉴스1', '한국경제TV']

# 크롤링 대상 언론사 데이터만 선택하여 새로운 데이터프레임 생성
filtered_df = df[df['article'].isin(values_to_keep)]

In [ ]:
print(len(filtered_df))
filtered_df.reset_index(inplace=True, drop=True) # 인덱스 정리
filtered_df

1920


,link,article,name,date
0,https://news.google.com/articles/CBMiOWh0dHBzO...,블로터,인스웨이브시스템즈,2023-08-13
1,https://news.google.com/articles/CBMiOWh0dHA6L...,비즈워치,인스웨이브시스템즈,2023-08-19
2,https://news.google.com/articles/CBMiJmh0dHBzO...,딜사이트,인스웨이브시스템즈,2023-08-04
3,https://news.google.com/articles/CBMiXWh0dHA6L...,더벨(thebell),인스웨이브시스템즈,2023-08-01
4,https://news.google.com/articles/CBMiOmh0dHBzO...,충청신문,인스웨이브시스템즈,2023-08-17
...,...,...,...,...
1915,https://news.google.com/articles/CBMiXGh0dHBzO...,더벨(thebell),티라유텍,2019-10-22
1916,https://news.google.com/articles/CBMiKmh0dHBzO...,이투데이,티라유텍,2019-10-16
1917,https://news.google.com/articles/CBMiKmh0dHBzO...,이투데이,미디어젠,2019-10-16
1918,https://news.google.com/articles/CBMiOmh0dHBzO...,서울파이낸스,미디어젠,2019-10-11


## 기사 내용 가져오기


*   신문사별 크롤링 함수가 다르기 때문에, if-elif-else 문을 사용하였습니다.



In [ ]:

for index, row in filtered_df.iterrows():
  if row['article']=='머니투데이':
    title,  content=mt(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='매일경제':
    title, content=mk(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='서울경제신문':
    title, content=sedaily(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='더벨(thebell)':
    title, content=thebell(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='한국경제':
    title, content=hankyung(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='이투데이':
    title,content=etoday(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='머니s' or row['article']=='머니S':
    title, content=moneys(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='아시아경제':
    title, content=asiae(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='서울파이낸스':
    title, content=seoulfn(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='딜사이트':
    title, content=dealsite(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='비즈워치':
    title,content=bizwatch(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='이코노미스트':
    title, content=economist(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='라이센스뉴스':
    title, content=license_news(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='시사저널e':
    title, content=sisa(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='충청신문':
    title,content=chungcheong(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='조선비즈':
    title, content=chosun(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='전국매일신문':
    title, content=jkmail(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='Investing.com':
    title, content=inv(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='영남일보':
    title,content=yn(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='CBC뉴스':
    title, content=cbc(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='블로터':
    title, content=blt(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='더팩트':
    title, content=thefact(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='아시아에이':
    title, content=asiaa(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='문화뉴스':
    title,content=culture(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='한겨레':
    title, content=hangyere(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  elif row['article']=='뉴스핌':
    title, content=pim(row['link'])

    filtered_df.at[index, 'title'] = title
    filtered_df.at[index, 'content'] = content
    print(f"{index}번째 index")

  else:
    pass


<ipython-input-164-608fb345d6bd>:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.at[index, 'title'] = title
<ipython-input-164-608fb345d6bd>:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.at[index, 'content'] = content


0번째 index
1번째 index
2번째 index
3번째 index
4번째 index
5번째 index
6번째 index
7번째 index
8번째 index
9번째 index
10번째 index
11번째 index
12번째 index
13번째 index
14번째 index
15번째 index
16번째 index
17번째 index
18번째 index
19번째 index
20번째 index
21번째 index
22번째 index
23번째 index
24번째 index
25번째 index
26번째 index
27번째 index
28번째 index
29번째 index
30번째 index
31번째 index
32번째 index
33번째 index
34번째 index
35번째 index
37번째 index
38번째 index
39번째 index
40번째 index
41번째 index
42번째 index
43번째 index
44번째 index
45번째 index
46번째 index
47번째 index
48번째 index
49번째 index
50번째 index
51번째 index
52번째 index
53번째 index
54번째 index
55번째 index
56번째 index
57번째 index
58번째 index
59번째 index
60번째 index
61번째 index
62번째 index
63번째 index
64번째 index
65번째 index
66번째 index
67번째 index
68번째 index
69번째 index
70번째 index
71번째 index
72번째 index
73번째 index
74번째 index
75번째 index
76번째 index
77번째 index
78번째 index
79번째 index
80번째 index
81번째 index
82번째 index
83번째 index
84번째 index
85번째 index
86번째 index
87번째 index
88번째 index
89번째 index
90번째 index
91번째 index
92번째 inde

In [ ]:
# csv로 저장
filtered_df.to_csv('[구글]1-15페이지_완성(찐).csv', index=False)